In [46]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [42]:
def time_to_seconds(time: str) -> int:
    h, m, s = map(int, time.split(':'))
    return s + m*60 + h*3600

In [51]:
def calc_angle(x1, y1, x2, y2, x3, y3):
    v1 = ((x2-x1), (y2-y1))
    v2 = ((x3-x2), (y3-y2))
    sqrt1, sqrt2 = np.sqrt([v1[0]**2+v1[1]**2, v2[0]**2+v2[1]**2])
    cos = (v1[0]*v2[0] + v1[1]*v2[1]) / (sqrt1*sqrt2)
    return (cos + 1) / 2

In [3]:
path = 'data\BadTracksHackaton1801.txt'

In [4]:
data = pd.read_csv(path, sep=' ', header=None, names=['time', 'id', 'latitude', 'longitude', 'elevation', 'code', 'name'])

In [5]:
data.head()

,time,id,latitude,longitude,elevation,code,name
0,0:00:01,8,49.722522,40.308650,11567,1025.0,TUM9102
1,0:00:11,8,49.737334,40.285857,11567,1025.0,TUM9102
2,0:00:21,8,49.752375,40.264201,11567,1025.0,TUM9102
3,0:00:31,8,49.767809,40.243869,11567,1025.0,TUM9102
4,0:00:41,8,49.785372,40.222412,11575,1025.0,TUM9102


In [62]:
grouped = data.groupby('id')